In [1]:
s3uri = "s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/generated/"

In [60]:
#查看生成的图片

In [2]:
!aws s3 ls $s3uri --recursive | wc -l

200


In [14]:
!aws s3 ls $s3uri --recursive

2023-06-19 09:45:44     415153 stable-diffusion-webui/generated/20230619014543-29fa9d13-5f2b-44ac-93e0-61f051223b44.png
2023-06-19 09:45:46     465319 stable-diffusion-webui/generated/20230619014545-7eab12be-e2ad-4ada-8c48-2c80e0134c4f.png
2023-06-19 09:45:48     474301 stable-diffusion-webui/generated/20230619014547-21132b0e-7a2e-4674-bab5-88214499f910.png
2023-06-19 09:45:50     447698 stable-diffusion-webui/generated/20230619014549-4aae6c88-5d4f-4531-8a3e-961bf8b5608a.png
2023-06-19 09:45:52     384482 stable-diffusion-webui/generated/20230619014551-a77ff5ee-9115-4a3d-84a5-4d86a8104358.png
2023-06-19 09:45:56     362030 stable-diffusion-webui/generated/20230619014555-59f1c1e3-79b4-45d6-97c2-b9567d539738.png
2023-06-19 09:46:00     513298 stable-diffusion-webui/generated/20230619014559-54865e9e-f05a-4610-8d34-f29f938dee3b.png
2023-06-19 09:46:04     474976 stable-diffusion-webui/generated/20230619014603-95ab088d-cd07-4fc5-bcba-8a2cde7c345d.png
2023-06-19 09:46:06     429796 stable-di

In [15]:
!aws s3 cp "$s3uri"20230619015249-d9332338-fb29-4d9b-b395-73022c1d447b.png .

download: s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/generated/20230619015249-d9332338-fb29-4d9b-b395-73022c1d447b.png to ./20230619015249-d9332338-fb29-4d9b-b395-73022c1d447b.png


In [132]:
#查看最新生成的图片

In [ ]:
import boto3
import urllib
from PIL import Image
import io

s3_resource = boto3.resource('s3')
def get_latest_object_uri(s3_uri):
    bucket_name, prefix = parse_s3_uri(s3_uri)
    
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        objects = response['Contents']
        objects.sort(key=lambda obj: obj['LastModified'], reverse=True)
        latest_object = objects[0]
        return f"s3://{bucket_name}/{latest_object['Key']}"
    
    return None

def _get_output_bucket_key(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    return bucket, key

def parse_s3_uri(s3_uri):
    parts = s3_uri.replace('s3://', '').split('/')
    bucket_name = parts[0]
    prefix = '/'.join(parts[1:])
    return bucket_name, prefix

# Gen ObjectURI
s3_uri = s3uri
latest_object_uri = get_latest_object_uri(s3_uri)
if latest_object_uri:
    print(latest_object_uri)
else:
    print("No objects found.")
    
bucket, key = _get_output_bucket_key(latest_object_uri)
obj = s3_resource.Object(bucket, key)
bytes = obj.get()['Body'].read()
image = Image.open(io.BytesIO(bytes))
image.show()


In [ ]:
#查看最近生成的随机的图片

In [ ]:
import boto3
import random
from PIL import Image
import requests
import io
from io import BytesIO

def get_latest_images(s3_uri, num_images):
    bucket_name, prefix = parse_s3_uri(s3_uri)

    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' in response:
        objects = response['Contents']
        objects.sort(key=lambda obj: obj['LastModified'], reverse=True)
        latest_objects = objects[:num_images]
        image_uris = [f"s3://{bucket_name}/{obj['Key']}" for obj in latest_objects]
        return image_uris

    return []

def _get_output_bucket_key(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    return bucket, key

def parse_s3_uri(s3_uri):
    parts = s3_uri.replace('s3://', '').split('/')
    bucket_name = parts[0]
    prefix = '/'.join(parts[1:])
    return bucket_name, prefix

# Example usage
s3_uri = s3uri
num_images = 5

latest_image_uris = get_latest_images(s3_uri, num_images)
if latest_image_uris:
    random_image_uri = random.choice(latest_image_uris)
    print("Randomly selected image URI:", random_image_uri)
    

    bucket, key = _get_output_bucket_key(random_image_uri)
    obj = s3_resource.Object(bucket, key)
    bytes = obj.get()['Body'].read()
    image = Image.open(io.BytesIO(bytes))
    image.show()
else:
    print("No images found.")


In [52]:
#查看输入输出

In [9]:
!aws s3 ls s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/asyncinvoke/in/ --recursive | wc -l

200


In [10]:
!aws s3 ls s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/asyncinvoke/out/ --recursive | wc -l

200


In [237]:
!aws s3 cp s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/asyncinvoke/out/e8e6e66c-e1a9-4471-a6a7-fcc5f18768cb.out /tmp | cat

download: s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/asyncinvoke/out/e8e6e66c-e1a9-4471-a6a7-fcc5f18768cb.out to ../../../../tmp/e8e6e66c-e1a9-4471-a6a7-fcc5f18768cb.out


In [ ]:
!cat /tmp/e8e6e66c-e1a9-4471-a6a7-fcc5f18768cb.out

In [ ]:
#删除图片及输入输出

In [ ]:
!aws s3 rm $s3uri --recursive

In [ ]:
!aws s3 rm s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/asyncinvoke/in/ --recursive
!aws s3 rm s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/asyncinvoke/out/ --recursive